# Inference with the model on openEO

This notebook demonstrates how to use a trained model for inference with onnxruntime in an udf.

In [1]:
# Make the connection and authenticate
import openeo
from utils import preprocess_sentinel2_data, postprocess_inference_data
from urllib.parse import urljoin


#Choose preferred backend
#conn = openeo.connect("https://openeo.cloud/").authenticate_oidc()
conn = openeo.connect("https://openeo.dataspace.copernicus.eu/").authenticate_oidc()


Authenticated using refresh token.


A workflow for processing Sentinel-2 Level-2A earth observation data using openEO in orden to obtain the desired data, compatible with the classifier


In [2]:
spatial_extent = {
                  "west": 16.341922243401232,
                  "east": 16.41408626641795,
                  "south": 47.96234024970855,
                  "north": 48.00780777335356
                }

temporal_extent = ["2023-05-01","2023-09-30"]

s2_cube = preprocess_sentinel2_data(conn, spatial_extent, temporal_extent)



Load in the model from the vito artifactory and the required dependencies and excecute the prediction job. The output is stored as a netCDF in the local directory

In [3]:
# Supply the model as a URL. The model is stored in artifactory
model_url = "https://artifactory.vgt.vito.be/artifactory/auxdata-public/photovoltaic/rf_1_median_depth_15.onnx"
udf = openeo.UDF.from_file(
    "udf_rf_onnx.py", 
    context={
        "model_url": model_url
    }
)

dependencies_url = "https://artifactory.vgt.vito.be/artifactory/auxdata-public/openeo/onnx_dependencies_1.16.3.zip"
dependencies_name = "onnx_deps"
full_url = urljoin(dependencies_url, f"#{dependencies_name}")

job_options = {
    "udf-dependency-archives": [
        full_url
    ],
}


# Reduce the bands dimesnion to a single prediction using the udf
prediction = s2_cube.reduce_bands(reducer=udf)

prediction_filtered = postprocess_inference_data(prediction, 3) 
# Post-process the data with an opening (erosion + dilation)


prediction_job = prediction_filtered.execute_batch(
    "./photovoltaic_prediction_median_summer_2023.tiff",
    job_options=job_options,
    title="photovoltaic_prediction_median_summer_2023"
)


0:00:00 Job 'j-2404057dacf047febf6f4e4cbe5c8e8f': send 'start'
0:00:21 Job 'j-2404057dacf047febf6f4e4cbe5c8e8f': created (progress N/A)
0:00:26 Job 'j-2404057dacf047febf6f4e4cbe5c8e8f': created (progress N/A)
0:00:33 Job 'j-2404057dacf047febf6f4e4cbe5c8e8f': created (progress N/A)
0:00:41 Job 'j-2404057dacf047febf6f4e4cbe5c8e8f': created (progress N/A)
0:00:51 Job 'j-2404057dacf047febf6f4e4cbe5c8e8f': created (progress N/A)
0:01:03 Job 'j-2404057dacf047febf6f4e4cbe5c8e8f': created (progress N/A)
0:01:19 Job 'j-2404057dacf047febf6f4e4cbe5c8e8f': running (progress N/A)
0:01:38 Job 'j-2404057dacf047febf6f4e4cbe5c8e8f': running (progress N/A)
0:02:02 Job 'j-2404057dacf047febf6f4e4cbe5c8e8f': running (progress N/A)
0:02:33 Job 'j-2404057dacf047febf6f4e4cbe5c8e8f': running (progress N/A)
0:03:10 Job 'j-2404057dacf047febf6f4e4cbe5c8e8f': running (progress N/A)
0:03:57 Job 'j-2404057dacf047febf6f4e4cbe5c8e8f': running (progress N/A)
0:04:56 Job 'j-2404057dacf047febf6f4e4cbe5c8e8f': running (pr

In [ ]:
import rioxarray
rioxarray.open_rasterio("./photovoltaic_prediction_median_summer_2023.tiff")[0].plot.imshow()

: 